colab 구글 드라이브 연동

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# DB 데이터 불러온 후 전처리

In [ ]:
import numpy as np
import pandas as pd

In [47]:
data = pd.read_csv("/content/drive/MyDrive/Zipbob_timeseries/pricedb예제data.csv", encoding='cp949') # 임시로 DB데이터 CSV로 저장 후 불러오기

In [48]:
data.head(10)

,itemnm,kindnm,regdate,price
0,팥,붉은 팥(국산)(1kg),1996-01-03,"3,375"
1,팥,붉은 팥(국산)(1kg),1996-01-04,"3,375"
2,팥,붉은 팥(국산)(1kg),1996-01-05,"3,375"
3,팥,붉은 팥(국산)(1kg),1996-01-08,"3,375"
4,팥,붉은 팥(국산)(1kg),1996-01-09,"3,375"
5,팥,붉은 팥(국산)(1kg),1996-01-10,"3,375"
6,팥,붉은 팥(국산)(1kg),1996-01-11,"3,375"
7,팥,붉은 팥(국산)(1kg),1996-01-12,"3,375"
8,팥,붉은 팥(국산)(1kg),1996-01-15,"3,375"
9,팥,붉은 팥(국산)(1kg),1996-01-16,"3,375"


In [ ]:
data.tail()

,itemnm,kindnm,regdate,price
335770,가지,가지(1kg),2011-04-21,"3,400"
335771,가지,가지(1kg),2011-04-22,"3,400"
335772,가지,가지(1kg),2011-04-25,"3,400"
335773,가지,가지(1kg),2011-04-26,"3,400"
335774,가지,가지(1kg),2011-04-27,"3,400"


In [ ]:
data.shape # 기존 데이터 335,775개

(335775, 4)

In [ ]:
data = data.drop_duplicates()

In [ ]:
data.shape # 중복제거 후 314,270개

(314270, 4)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 314270 entries, 0 to 331753
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   itemnm   314270 non-null  object
 1   kindnm   314270 non-null  object
 2   regdate  314270 non-null  object
 3   price    314270 non-null  object
dtypes: object(4)
memory usage: 12.0+ MB


## 시계열 가격예측을 위한 카테고리 생성
- itemnm-kindnm조합 추출하기 83개
- item마다 kindnm이 여러개 있는 경우가 있고, item마다 동일한 단위 '국산(1kg)'처럼 중첩되므로 item으로만 분류하지 않음(EDA코드확인)

In [ ]:
itemlist = list(data['itemnm'].unique())

In [ ]:
itemkind=[] # 리스트 생성

In [ ]:
for i in itemlist :
    dataset = data[data['itemnm']==i]
    kind_unique = list(dataset['kindnm'].unique())
    for k in range(len(kind_unique)) :
        itemkind.append([i, kind_unique[k]])

In [ ]:
itemkind[:10], len(itemkind)

([['팥', '붉은 팥(국산)(1kg)'],
  ['녹두', '국산(1kg)'],
  ['고구마', '밤(1kg)'],
  ['양배추', '양배추(1kg)'],
  ['시금치', '시금치(1kg)'],
  ['얼갈이배추', '얼갈이배추(1kg)'],
  ['수박', '수박(1개)'],
  ['참외', '참외(1kg)'],
  ['토마토', '토마토(1kg)'],
  ['딸기', '딸기(1kg)']],
 83)

In [ ]:
data['itemkind'] = data['itemnm']+'-'+data['kindnm']

## 결측치 '-' 처리하기
햇양파같은 제철야채 식품은 특정 기간에 price가 '-'으로 등록됨. 따라서 0 처리

In [ ]:
data1 = data.copy()

In [ ]:
data1 = data1.replace('-', '0')

## price가격 object타입 변환 : , 없애고 숫자로

In [ ]:
data1['price'] = data1['price'].str.replace(',', '')
data1['price'] = data1['price'].astype('int64')

#### 데이터 전처리 완료.

#### 저장해서 다시 불러오거나 바로 진행

In [ ]:
# data1.to_csv('/경로설정/농수산DB기본전처리.csv', index=False, encoding='cp949')
# data = pd.read_csv('/content/drive/MyDrive/Zipbob_timeseries/농수산DB기본전처리.csv', encoding='cp949')

In [ ]:
data = data1.copy()

In [ ]:
data.head()

,itemnm,kindnm,regdate,price,itemkind
0,팥,붉은 팥(국산)(1kg),1996-01-03,3375,팥-붉은 팥(국산)(1kg)
1,팥,붉은 팥(국산)(1kg),1996-01-04,3375,팥-붉은 팥(국산)(1kg)
2,팥,붉은 팥(국산)(1kg),1996-01-05,3375,팥-붉은 팥(국산)(1kg)
3,팥,붉은 팥(국산)(1kg),1996-01-08,3375,팥-붉은 팥(국산)(1kg)
4,팥,붉은 팥(국산)(1kg),1996-01-09,3375,팥-붉은 팥(국산)(1kg)


# 가격 예측 진행

## 필요한 모듈 설치

In [ ]:
# !pip install tzlocal
import tzlocal
# !pip install rpy2
import rpy2
!pip install pmdarima==1.5.3 # arima 용 설치
import pmdarima # arima 용 설치
import statsmodels
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt

(arima, ets용 forecast 설치. 오래걸림)

In [ ]:
# from rpy2.robjects.packages import importr
# utils = importr('utils')
# utils.install_packages('forecast')

## 시계열용 datetime 변환

In [ ]:
traindf = data.copy()

In [ ]:
traindf['regdate'] = pd.to_datetime(traindf['regdate'],format="%Y-%m-%d")

In [ ]:
traindf = traindf.drop(['itemnm', 'kindnm'], axis=1)

# fbprophet 세팅

In [ ]:
!python --version

Python 3.7.12


In [ ]:
!pip install pystan==2.19.1.1
!pip install prophet

In [ ]:
# 관련된 오류나면 버전 바꿔서 실행하기
# !pip install cmdstanpy==0.9.5

  Attempting uninstall: cmdstanpy
    Found existing installation: cmdstanpy 0.9.68
    Uninstalling cmdstanpy-0.9.68:
      Successfully uninstalled cmdstanpy-0.9.68
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
prophet 1.0.1 requires cmdstanpy==0.9.68, but you have cmdstanpy 0.9.5 which is incompatible.


In [ ]:
import fbprophet
print(fbprophet.__version__)

0.7.1


In [ ]:
from fbprophet import Prophet

## fbprophet에 맞게 컬럼명 ds(날짜), y(예측값)로 지정

In [ ]:
train_rename = traindf.rename(columns = {'regdate' : 'ds', 'price':'y'})

In [ ]:
train_rename

,ds,y,itemkind
0,1996-01-03,3375,팥-붉은 팥(국산)(1kg)
1,1996-01-04,3375,팥-붉은 팥(국산)(1kg)
2,1996-01-05,3375,팥-붉은 팥(국산)(1kg)
3,1996-01-08,3375,팥-붉은 팥(국산)(1kg)
4,1996-01-09,3375,팥-붉은 팥(국산)(1kg)
...,...,...,...
331749,2020-12-24,8000,피마늘-한지(1kg)
331750,2020-12-28,8000,피마늘-한지(1kg)
331751,2020-12-29,8000,피마늘-한지(1kg)
331752,2020-12-30,8000,피마늘-한지(1kg)


## 시계열 예측 대상 날짜구간 생성하기
DB의 최근 날짜에 맞춰서 +28일 데이터 자동 생성

In [ ]:
from datetime import datetime, timedelta
pred_dayslist = []
for k in range(1, 29) :
  a = train_rename['ds'].max()
  pred_dayslist.append(a+timedelta(days=k))

In [ ]:
new_data = pd.DataFrame()
# 예측을 위한 ds컬럼 생성
new_data['ds'] = pred_dayslist

# fbprophet 모델링

In [ ]:
item_pred_list = []
itemkind_name_list = []

- 야채 itemkind마다 모델 생성 -> 모델링하므로 colab에서 20분 이상 소요
- 전체 django프로젝트 실행 시 메모리할당량 초과되면 colab에서 별도로 예측 데이터 추출 후 활용하기

In [ ]:
for i in tqdm(train_rename.itemkind.unique()): # 진행상황 표시하는 tqdm
  selected_train = train_rename[train_rename['itemkind']==i] # 각 야채카테고리별 데이터셋 생성

  model = Prophet(
    # trend
    changepoint_prior_scale=0.5,
    # seasonality
    weekly_seasonality=10,
    yearly_seasonality=10,
    daily_seasonality=True,
    seasonality_mode='multiplicative'
  )
  model.fit(selected_train)

  forecast = model.predict(new_data)
  y_pred = forecast['yhat'].values
  print(i, "예측값완료")
  
  itemkind_name_list.append(i)
  item_pred_list.append(y_pred)

  1%|          | 1/83 [00:26<36:07, 26.43s/it]

팥-붉은 팥(국산)(1kg) 예측값완료


  2%|▏         | 2/83 [01:05<45:56, 34.03s/it]

녹두-국산(1kg) 예측값완료


  4%|▎         | 3/83 [01:29<39:07, 29.34s/it]

...

상추-청(1kg) 예측값완료


 99%|█████████▉| 82/83 [26:30<00:11, 11.80s/it]

오이-가시계통(1kg) 예측값완료


100%|██████████| 83/83 [27:12<00:00, 19.66s/it]

무-가을(1kg) 예측값완료


In [50]:
# itemkind_name_list
len(item_pred_list)

83

In [51]:
item_pred_list[:3]

[array([11867.98743147, 11893.57496514, 11671.22080707, 11846.00337365,
        11827.61073179, 11861.32249627, 11839.89594215, 11815.88532807,
        11833.96937949, 11604.78355628, 11774.07119211, 11750.87223216,
        11780.65668816, 11755.98064202, 11729.3410385 , 11745.33375015,
        11514.15284419, 11682.06332602, 11657.34385696, 11685.57569596,
        11659.0714603 , 11630.26709413, 11643.71100111, 11409.22955077,
        11573.7194329 , 11544.84117896, 11568.45931591, 11536.83863143]),
 array([15597.81667847, 15670.74212529, 14243.16937803, 15582.48269036,
        15592.53998337, 15608.85446727, 15591.42119102, 15556.15667134,
        15619.37687616, 14180.12232909, 15513.52370695, 15516.08601486,
        15525.75955983, 15502.4862613 , 15462.22252192, 15521.49806589,
        14076.46821827, 15409.95556109, 15411.0632441 , 15420.13519242,
        15397.02187059, 15357.68592939, 15418.84131063, 13973.73202851,
        15312.84916676, 15317.87260323, 15331.51976331, 15313.

In [ ]:
y_pred # 가장 마지막 야채컬럼 예측값(0은 마이너스)

array([  1.99147791,  -7.77831435,  18.16211624,   4.18505882,
         2.45468968,   0.54684773,   1.74186501,  -0.822578  ,
       -11.02966751,  14.55991498,   0.2755744 ,  -1.65610161,
        -3.64816356,  -2.39536218,  -4.7425493 , -14.55251854,
        11.67305056,  -1.77446757,  -2.60327007,  -3.21488651,
        -0.28146077,  -0.63570816,  -8.12864206,  20.78495717,
        10.34269332,  12.88213268,  15.99011093,  22.99032781])

# 농수산별 예측값 통합 DataFrame 생성

In [ ]:
formerge = pd.DataFrame()
formerge['pred_date'] = None
formerge['itemkind'] = None
formerge['price_pred'] = None

for i in range(len(itemkind_name_list)) :
  formerge1 = pd.DataFrame()
  formerge1['pred_date'] = new_data['ds']
  formerge1['itemkind'] = itemkind_name_list[i]
  formerge1['price_pred'] = item_pred_list[i]
  formerge1['itemnm'] = itemkind_name_list[i].split("-")[0]
  formerge1['kindnm'] = itemkind_name_list[i].split("-")[1]
  formerge = pd.concat([formerge,formerge1],axis=0)

In [ ]:
formerge = formerge.reset_index()
formerge.drop('index', axis=1, inplace=True)

In [ ]:
formerge
# formerge = formerge.drop_duplicates()

,pred_date,itemkind,price_pred,itemnm,kindnm
0,2021-09-17,팥-붉은 팥(국산)(1kg),11867.987431,팥,붉은 팥(국산)(1kg)
1,2021-09-18,팥-붉은 팥(국산)(1kg),11893.574965,팥,붉은 팥(국산)(1kg)
2,2021-09-19,팥-붉은 팥(국산)(1kg),11671.220807,팥,붉은 팥(국산)(1kg)
3,2021-09-20,팥-붉은 팥(국산)(1kg),11846.003374,팥,붉은 팥(국산)(1kg)
4,2021-09-21,팥-붉은 팥(국산)(1kg),11827.610732,팥,붉은 팥(국산)(1kg)
...,...,...,...,...,...
2319,2021-10-10,무-가을(1kg),20.784957,무,가을(1kg)
2320,2021-10-11,무-가을(1kg),10.342693,무,가을(1kg)
2321,2021-10-12,무-가을(1kg),12.882133,무,가을(1kg)
2322,2021-10-13,무-가을(1kg),15.990111,무,가을(1kg)


### 마이너스 값 0처리

In [ ]:
formerge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2324 entries, 0 to 2323
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   pred_date   2324 non-null   datetime64[ns]
 1   itemkind    2324 non-null   object        
 2   price_pred  2324 non-null   float64       
 3   itemnm      2324 non-null   object        
 4   kindnm      2324 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 90.9+ KB


In [ ]:
def cleansing(x) :
  if x < 0 : x = 0
  return x

In [ ]:
formerge['price_pred'] = formerge['price_pred'].apply(cleansing)

In [ ]:
formerge

,pred_date,itemkind,price_pred,itemnm,kindnm
0,2021-09-17,팥-붉은 팥(국산)(1kg),11867.987431,팥,붉은 팥(국산)(1kg)
1,2021-09-18,팥-붉은 팥(국산)(1kg),11893.574965,팥,붉은 팥(국산)(1kg)
2,2021-09-19,팥-붉은 팥(국산)(1kg),11671.220807,팥,붉은 팥(국산)(1kg)
3,2021-09-20,팥-붉은 팥(국산)(1kg),11846.003374,팥,붉은 팥(국산)(1kg)
4,2021-09-21,팥-붉은 팥(국산)(1kg),11827.610732,팥,붉은 팥(국산)(1kg)
...,...,...,...,...,...
2319,2021-10-10,무-가을(1kg),20.784957,무,가을(1kg)
2320,2021-10-11,무-가을(1kg),10.342693,무,가을(1kg)
2321,2021-10-12,무-가을(1kg),12.882133,무,가을(1kg)
2322,2021-10-13,무-가을(1kg),15.990111,무,가을(1kg)


In [ ]:
formerge['price_pred'].min()

0.0

# 최종 결과 저장

저장 경로 바꿔서 진행

In [ ]:
formerge.to_csv('/content/drive/MyDrive/Zipbob_timeseries/kamis_4weeks_predict.csv', index=False, encoding='cp949')